### MNIST

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

# move to GPU (if applicable)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
# choose network configuration and dataset
backdoor = Backdoor(VggNet, config='16-layer', channels=1, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.MNIST, ImageEntity)

### Poison setup

In [3]:
# create patches, target labels, and define transformations/merges 
patches = ImagePatch((9, 9), 1, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = ([ExpandTransform(1)], [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
# poison 20% of the data
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
# train the base DNN & the trojan DNN 
loss = backdoor.train(2, epochs=1, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/1 | 24.99% | Loss: 0.2832 | Samples trained: 14992/60000
Epoch 1/1 | 49.97% | Loss: 0.1837 | Samples trained: 29984/60000
Epoch 1/1 | 74.96% | Loss: 0.1452 | Samples trained: 44976/60000
Epoch 1/1 | 99.95% | Loss: 0.1235 | Samples trained: 59968/60000
Epoch 1 complete | Loss: 0.1234
Training complete | Net Average Loss: 0.1234 | Total epochs: 1
Training started


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 9.78 GiB total capacity; 1.61 GiB already allocated; 109.94 MiB free; 1.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Evaluating Backdoor

In [ ]:
# evulate backdoor based on VA & ASR
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 98.64% | Trojan 98.62% | Difference -0.02%
Base Accuracy on Poison 0.53% | Attack Success Rate (ASR): 95.07%
Average Tensor Distance: 234.22 | Net Tensor Difference 14053063.53


### Creating a Defense

In [ ]:
# init the defense
defense = Defense(backdoor)

### Detection setup

In [ ]:
# get base & trojan DNN, both clean & poisoned entitysets, sizes ranges to create synthetic patch
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [ ]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for base DNN
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 98.54% | Original Accuracy 98.64% | Difference -0.10%
Patch size (5, 5) | Synthetic Poison Accuracy 97.91% | Original Accuracy 98.64% | Difference -0.73%
Patch size (7, 7) | Synthetic Poison Accuracy 96.81% | Original Accuracy 98.64% | Difference -1.83%
Patch size (9, 9) | Synthetic Poison Accuracy 95.20% | Original Accuracy 98.64% | Difference -3.44%
Patch size (11, 11) | Synthetic Poison Accuracy 92.55% | Original Accuracy 98.64% | Difference -6.09%
Average Accuracy 96.20% | Average Difference -2.44% | Lowest Score: 92.55% | Likihood of Backdoor: Low


### Detection Trojan

In [ ]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for trojan DNN
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 98.43% | Original Accuracy 98.62% | Difference -0.19%
Patch size (5, 5) | Synthetic Poison Accuracy 96.84% | Original Accuracy 98.62% | Difference -1.78%
Patch size (7, 7) | Synthetic Poison Accuracy 84.93% | Original Accuracy 98.62% | Difference -13.69%
Patch size (9, 9) | Synthetic Poison Accuracy 69.42% | Original Accuracy 98.62% | Difference -29.20%
Patch size (11, 11) | Synthetic Poison Accuracy 69.28% | Original Accuracy 98.62% | Difference -29.34%
Average Accuracy 83.78% | Average Difference -14.84% | Lowest Score: 69.28% | Likihood of Backdoor: High


### Blocking Patches setup

In [ ]:
# create patch to compare variances with in parts of images (same size as patch), define samples to block
patch = ImagePatch((10, 10), 1, 'random')
n = 5000

### Blocking

In [ ]:
# block poisoned images with average rgb values the same dimensions as the patch
blockedloader = defense.block(poisontest, patch, n=n)

### Testing after Block Base

In [ ]:
# retest after blocking for base DNN
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
24.84% Testing complete | Loss: 4.7976 | Accuracy: 0.0385
49.68% Testing complete | Loss: 4.7474 | Accuracy: 0.0409
74.52% Testing complete | Loss: 4.7837 | Accuracy: 0.0393
99.36% Testing complete | Loss: 4.7441 | Accuracy: 0.0439
Testing complete | Loss: 4.7426 | Accuracy: 4.38%


### Testing after Block Trojan

In [ ]:
# retest after blocking for trojan DNN
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
24.84% Testing complete | Loss: 1.0572 | Accuracy: 0.6907
49.68% Testing complete | Loss: 1.1024 | Accuracy: 0.6755
74.52% Testing complete | Loss: 1.1047 | Accuracy: 0.6760
99.36% Testing complete | Loss: 1.0982 | Accuracy: 0.6773
Testing complete | Loss: 1.1007 | Accuracy: 67.72%


### Defense by retraining

In [ ]:
# get a clean dataloader, reset the defense net module, retrain the defense net model (og trojan model)
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=1, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/1 | 24.96% | Loss: 0.2696 | Samples trained: 14976/60000
Epoch 1/1 | 49.92% | Loss: 0.2679 | Samples trained: 29952/60000
Epoch 1/1 | 74.88% | Loss: 0.2661 | Samples trained: 44928/60000
Epoch 1/1 | 99.84% | Loss: 0.2626 | Samples trained: 59904/60000
Epoch 1 complete | Loss: 0.2626
Training complete | Net Average Loss: 0.2626 | Total epochs: 1


### Evaluation after retraining

In [ ]:
# re-evaluate the trojan DNN
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 98.64% | Trojan 98.62% | Defense 90.34%
Accuracy on Posion | Base 0.53% | Defense 84.98% | Trojan ASR 95.07%
Difference from Baseline | Trojan -0.02% | Defense -8.30%
Defense Effectiveness | 10.09% decrease in ASR


In [ ]:
defense.view_named_modules()

avgpool: AdaptiveAvgPool2d(output_size=(7, 7))
drop: Dropout(p=0.5, inplace=False)
fc1: Linear(in_features=25088, out_features=4096, bias=True)
fc2: Linear(in_features=4096, out_features=4096, bias=True)
fc3: Linear(in_features=4096, out_features=10, bias=True)
layers: Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=1, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running

### Pruning setup

In [ ]:
# determine layers to prune, decide percent of neurons to prune, reset the defense (original trojan DNN)
layers = ['layers.28', 'layers.26', 'layers.24', 'layers.21']
amount = 0.5
defense.reset()

### Pruning based on l-Infinity norm of weights from layers

In [ ]:
# prune 50% the weights of last 4 convolutional layers based on the l-Infinity norm
defense.prune(layers, amount)

### Evaluation after prune

In [ ]:
# evaluate the model after pruning
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 98.64% | Trojan 98.62% | Defense 9.80%
Accuracy on Posion | Base 0.53% | Defense 10.09% | Trojan ASR 95.07%
Difference from Baseline | Trojan -0.02% | Defense -88.84%
Defense Effectiveness | 84.98% decrease in ASR


### Retraining to recover

In [ ]:
# retrain to recover
loss = defense.retrain(dataloader, epochs=1, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/1 | 24.96% | Loss: 2.4359 | Samples trained: 14976/60000
Epoch 1/1 | 49.92% | Loss: 2.4350 | Samples trained: 29952/60000
Epoch 1/1 | 74.88% | Loss: 2.4341 | Samples trained: 44928/60000
Epoch 1/1 | 99.84% | Loss: 2.4323 | Samples trained: 59904/60000
Epoch 1 complete | Loss: 2.4324
Training complete | Net Average Loss: 2.4324 | Total epochs: 1


### Evaluation after retraining

In [ ]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 98.64% | Trojan 98.62% | Defense 8.92%
Accuracy on Posion | Base 0.53% | Defense 9.82% | Trojan ASR 95.07%
Difference from Baseline | Trojan -0.02% | Defense -89.72%
Defense Effectiveness | 85.25% decrease in ASR
